In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

from matrix_utils import (arr_info,
                          make_linmap)
from vis_utils import (vis_rgb_cube,
                       vis_hists,
                       vis_pair,
                       vis_surface)

from wavelet_utils import (make_haar_matrix,
                           make_random_basis,
                           make_klt_basis,
                           make_dct_matrix,
                           make_standard_matrix,
                           vis_blocks)

from skimage.util import view_as_blocks
from skimage.util import montage
from skimage.transform import resize, rescale
from ipywidgets import VBox, HBox, FloatSlider

In [2]:
I = plt.imread('../dip_pics/machu.png')
I = I[...,:3]
I = ((I - I.min()) / (I.max() - I.min()) * 255).astype('uint8') # Normalize the figure before processing
vis_hists(I)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
block_shape = (8,8,3)
view = view_as_blocks(I, block_shape)

In [4]:
D = make_dct_matrix(8)
Q = np.array([[16, 11, 10, 16, 24, 40, 51, 61],
              [12, 12, 14, 19, 26, 58, 60, 55],
              [14, 13, 16, 24, 40, 57, 69, 56],
              [14, 17, 22, 29, 51, 87, 80, 62],
              [18, 22, 37, 56, 68, 109, 103, 77],
              [24, 35, 55, 64, 81, 104, 113, 92],
              [49, 64, 78, 87, 103, 121, 120, 101],
              [72, 92, 95, 98, 112, 100, 103, 99]])

In [5]:
block_view = np.reshape(view, [view.shape[0]*view.shape[1]] + list(block_shape))

In [43]:
def quantize(m):
    global block_view
    mod_block_view = (np.zeros_like(block_view)).astype('float')
    
    for i, block in enumerate(block_view):
        Ir = (np.zeros_like(block)).astype('float')
        for chan in range(3):
            block_temp = block[...,chan].astype('float')-128
            T = np.matmul(D, np.matmul(block_temp,D.transpose()))
            Tq = np.round(T / (m*Q))
            Ir[...,chan] = np.matmul(D.transpose(), np.matmul(Tq*m*Q, D)) + 128
            
        mod_block_view[i] = Ir
        
    block_grid = np.clip(montage(mod_block_view, grid_shape = view.shape[:2], multichannel=True, padding_width = 0, fill=[1,1,1]),0,255)
    return block_grid/255, T, Tq
    

In [46]:
plt.ioff()
plt.clf()

mult_slider = FloatSlider(
    orientation = 'horizontal',
    value = 1.00,
    min = 0.20,
    max = 5.00,
    step = 0.01,
    description = 'Quantization Multiples:'
)

fig_args = {'num':' ', 'frameon':True, 'sharex':True, 'sharey':True}
fig, ax = plt.subplots(1,3, figsize=(8,4), **fig_args)

Original = ax[0].imshow(I[240:320,280:320,:])
ax[0].set_title('Original Image')
Reconstruct = ax[1].imshow(I[240:320,280:320,:])
text = ax[1].set_title('Quantized Reconstruction')
Coeff = ax[2].imshow(I[240:320,280:320,:])
DCTQ = ax[2].set_title('DCT coefficients')

def update_image(change):
    global Original, Reconstruct, text
    Iq, T, Tq = quantize(change.new)
    Reconstruct.set_array(Iq[240:320,280:320,:])
    text.set_text(f'Quantized Reconstruction \n with multiple: { format(change.new)}')
    Coeff.set_array(Tq)
    DCTQ.set_text(f'DCT Coeff with multiple \n of { format(change.new)}')
    fig.canvas.draw()
    fig.canvas.flush_events()

mult_slider.observe(update_image, names = "value")

VBox([mult_slider, fig.canvas])